In [7]:

from pylabnet.utils.zi_hdawg_pulseblock_handler.zi_hdawg_pb_handler import DIOPulseBlockHandler

import pylabnet.utils.pulseblock.pulse as po
import pylabnet.utils.pulseblock.pulse_block as pb
from pylabnet.utils.pulseblock.pb_iplot import iplot
from pylabnet.utils.pulseblock.pb_sample import pb_sample
from pylabnet.utils.logging.logger import LogClient

from pylabnet.hardware.awg.zi_hdawg import Driver, Sequence, AWGModule




In [8]:

dev_id = 'dev8227'

# Instantiate logger.
logger = LogClient(
    host='140.247.189.82',
    port=20190,
    module_tag=f'ZI HDAWG {dev_id}'
)

# Instanciate HDAWG driver.
hd = Driver(dev_id, logger=logger)


In [132]:

scaling = 1
laser_2 = "sasha"
laser_1 = "toptica"


t_start = 0
delta_t_laser_1 = 2e-3 * scaling
gate_buffer = 1e-6
delta_t_gate = 1e-7
delta_t_laser_2 = 2e-3* scaling
delta_t_start = 1e-7


# Define the pulse sequence
scan_pulse = pb.PulseBlock(
    p_obj_list=[
        po.PTrue(ch=laser_1, t0=t_start, dur=delta_t_laser_1),
        po.PTrue(ch=laser_2, t0=t_start+delta_t_laser_1+2*gate_buffer, dur=delta_t_laser_2),
        po.PTrue(ch='gate1', t0=t_start+delta_t_laser_1+gate_buffer, dur=delta_t_gate+gate_buffer),
    ]
)

In [133]:
iplot(scan_pulse)

In [134]:

assignmentdict = {
    "sasha" : 1,
    "toptica" : 0,
    "gate1": 2
}


# Instanciate pulseblock handler.
pb_handler = DIOPulseBlockHandler(
    pb = scan_pulse,
    assignment_dict=assignmentdict,
    hd=hd
)

In [135]:
codewords = pb_handler.gen_codewords()
from pylabnet.gui.igui.iplot import SingleTraceFig
import numpy as np


In [136]:

staggered_trace = SingleTraceFig()

staggered_trace.set_data(
    y_ar=codewords,
    x_ar = np.arange(len(codewords)) * 1 / (300e6)
)

iplot(scan_pulse)
staggered_trace.show()

FigureWidget({
    'data': [{'mode': 'lines',
              'type': 'scatter',
              'uid': '9615e3e2-…

In [137]:
dio_vals, waittimes = pb_handler.zip_dio_commands(codewords)

In [138]:
dio_vals

array([1, 0, 4, 6, 2, 0], dtype=int64)

In [139]:
waittimes * 1/300e6

array([1.99999667e-03, 1.00000000e-06, 9.96666667e-07, 1.06666667e-07,
       1.99990000e-03, 6.66666667e-09])

In [129]:
waittimes * 1/300e6

array([1.99999667e-03, 1.00000000e-06, 9.96666667e-07, 5.06666667e-07,
       1.99950000e-03, 6.66666667e-09])

In [140]:
waittimes

array([599999,    300,    299,     32, 599970,      2], dtype=int64)

In [141]:
pb_handler.get_dio_sequence()

'setDIO(1);wait(599995);setDIO(0);wait(296);setDIO(4);wait(295);setDIO(6);wait(28);setDIO(2);wait(599966);setDIO(0);wait(0);'